In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [2]:
import numpy as np
import tensorflow as tf
import json

In [3]:
with open('dataset-bpe.json') as fopen:
    data = json.load(fopen)

In [4]:
train_X = data['train_X']
train_Y = data['train_Y']
test_X = data['test_X']
test_Y = data['test_Y']

In [5]:
EOS = 2
GO = 1
vocab_size = 32000

In [6]:
train_Y = [i + [2] for i in train_Y]
test_Y = [i + [2] for i in test_Y]

In [7]:
from tensor2tensor.utils import beam_search

def pad_second_dim(x, desired_size):
    padding = tf.tile([[[0.0]]], tf.stack([tf.shape(x)[0], desired_size - tf.shape(x)[1], tf.shape(x)[2]], 0))
    return tf.concat([x, padding], 1)

class Translator:
    def __init__(self, size_layer, num_layers, embedded_size, learning_rate):
        
        def cells(size_layer=size_layer, reuse=False):
            return tf.nn.rnn_cell.GRUCell(size_layer,reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype = tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype = tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        embeddings = tf.Variable(tf.random_uniform([vocab_size, embedded_size], -1, 1))
        
        def forward(x, y, reuse = False):
            batch_size = tf.shape(x)[0]
            X_seq_len = tf.count_nonzero(x, 1, dtype = tf.int32)
            Y_seq_len = tf.count_nonzero(y, 1, dtype = tf.int32)
            with tf.variable_scope('model',reuse=reuse):
                encoder_embedded = tf.nn.embedding_lookup(embeddings, x)
                decoder_embedded = tf.nn.embedding_lookup(embeddings, y)
                for n in range(num_layers):
                    (out_fw, out_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(
                        cell_fw = cells(size_layer // 2),
                        cell_bw = cells(size_layer // 2),
                        inputs = encoder_embedded,
                        sequence_length = self.X_seq_len,
                        dtype = tf.float32,
                        scope = 'bidirectional_rnn_%d'%(n))
                    encoder_embedded = tf.concat((out_fw, out_bw), 2)
                    
                bi_state = tf.concat((state_fw,state_bw), -1)
                last_state = tuple([bi_state] * num_layers)
                last_output = tf.concat((out_fw,out_bw), -1)

            with tf.variable_scope("decoder",reuse=reuse):
                
                attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units = size_layer, 
                                                                    memory = last_output)
                rnn_cells = tf.contrib.seq2seq.AttentionWrapper(
                    cell = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)]), 
                    attention_mechanism = attention_mechanism,
                    attention_layer_size = size_layer)
                
                initial_state = rnn_cells.zero_state(batch_size, tf.float32).clone(cell_state=last_state)
                outputs, _ = tf.nn.dynamic_rnn(rnn_cells, decoder_embedded, 
                                               sequence_length=Y_seq_len,
                                               initial_state = initial_state,
                                               dtype = tf.float32)
                
                return tf.layers.dense(outputs,vocab_size)
            
        main = tf.strided_slice(self.X, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        self.training_logits = forward(self.X, decoder_input, reuse = False)
        
        self.training_logits = self.training_logits[:, :tf.reduce_max(self.Y_seq_len)]
        self.training_logits = pad_second_dim(self.training_logits, tf.reduce_max(self.Y_seq_len))
            
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        initial_ids = tf.fill([batch_size], GO)
        def symbols_to_logits(ids):
            x = tf.contrib.seq2seq.tile_batch(self.X, 1)
            logits = forward(x, ids, reuse = True)
            return logits[:, tf.shape(ids)[1]-1, :]
        
        final_ids, final_probs, _ = beam_search.beam_search(
            symbols_to_logits,
            initial_ids,
            1,
            tf.reduce_max(self.X_seq_len),
            vocab_size,
            0.0,
            eos_id = EOS)
        
        self.fast_result = final_ids

In [8]:
size_layer = 512
num_layers = 2
embedded_size = 256
learning_rate = 1e-3
batch_size = 128
epoch = 20

In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator(size_layer, num_layers, embedded_size, learning_rate)
sess.run(tf.global_variables_initializer())

Instructions for updating:
reduction_indices is deprecated, use axis instead
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/2018090

In [10]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [11]:
batch_x = pad_sequences(train_X[:10], padding='post')
batch_y = pad_sequences(train_Y[:10], padding='post')

sess.run([model.fast_result, model.cost, model.accuracy], 
         feed_dict = {model.X: batch_x, model.Y: batch_y})

[array([[[    1, 18052, 18052, 19804, 19804, 19804,  4313,  6055,  6155,
           9694,  9694, 11207, 11207,   347,  1959,  1959,  1959,  8028,
           8028,  8028,  8028, 15119, 15119, 15119,  3252,  3252,  3252,
           3252,  9799,  9799,  9799, 26138, 20918, 20918, 20918, 22175,
          26691]],
 
        [[    1, 25834, 25834, 20262, 20262, 26498,   272,   272, 31970,
            272,   272,   272,   272, 16273, 16273, 16273, 15479, 15479,
          15479,  9500,  1841,  1841,  3809,  3809,  3809,  3809,  9120,
          14024, 14024, 14024, 14024, 14024, 14024, 10565, 10565, 10565,
          10565]],
 
        [[    1,   234,   234, 31195,  1235,  1235, 14508, 14508, 14508,
          14508, 24206, 24206, 27747, 21990, 21990, 21448, 31056, 29858,
          17227, 17227, 29858, 19343, 15649, 15649,  9389,  9389, 16711,
          16711, 20530, 20530, 20530, 28698, 28698,  3466,  3466,  3466,
           3466]],
 
        [[    1, 24476,  5076, 17148, 15651, 15651, 19435, 20

In [12]:
import tqdm

for e in range(epoch):
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = pad_sequences(train_X[i : index], padding='post')
        batch_y = pad_sequences(train_Y[i : index], padding='post')
        feed = {model.X: batch_x,
                model.Y: batch_y}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    
    pbar = tqdm.tqdm(
        range(0, len(test_X), batch_size), desc = 'minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = pad_sequences(test_X[i : index], padding='post')
        batch_y = pad_sequences(test_Y[i : index], padding='post')
        feed = {model.X: batch_x,
                model.Y: batch_y,}
        accuracy, loss = sess.run([model.accuracy,model.cost],
                                    feed_dict = feed)

        test_loss.append(loss)
        test_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('epoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))
    print('epoch %d, testing avg loss %f, testing avg acc %f'%(e+1,
                                                              np.mean(test_loss),np.mean(test_acc)))

minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 1, training avg loss 6.644330, training avg acc 0.129178
epoch 1, testing avg loss 6.014792, testing avg acc 0.152007


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 2, training avg loss 5.684443, training avg acc 0.159240
epoch 2, testing avg loss 5.788021, testing avg acc 0.155117


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

...

epoch 15, training avg loss 3.965926, training avg acc 0.243884
epoch 15, testing avg loss 6.033653, testing avg acc 0.159831


minibatch loop:  47%|████▋     | 739/1563 [05:54<07:09,  1.92it/s, accuracy=0.204, cost=4.46]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 18, training avg loss 4.055176, training avg acc 0.236952
epoch 18, testing avg loss 6.173583, testing avg acc 0.150324


minibatch loop:   4%|▍         | 69/1563 [01:02<14:22,  1.73it/s, accuracy=0.202, cost=4.38]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

minibatch loop: 100%|██████████| 40/40 [00:08<00:00,  4.99it/s, accuracy=0.151, cost=6.31]

epoch 20, training avg loss 3.881600, training avg acc 0.255196
epoch 20, testing avg loss 6.484758, testing avg acc 0.139679


In [13]:
from tensor2tensor.utils import bleu_hook

In [14]:
results = []
for i in tqdm.tqdm(range(0, len(test_X), batch_size)):
    index = min(i + batch_size, len(test_X))
    batch_x = pad_sequences(test_X[i : index], padding='post')
    feed = {model.X: batch_x}
    p = sess.run(model.fast_result,feed_dict = feed)[:,0,:]
    result = []
    for row in p:
        result.append([i for i in row if i > 3])
    results.extend(result)

100%|██████████| 40/40 [04:34<00:00,  6.86s/it]


In [15]:
rights = []
for r in test_Y:
    rights.append([i for i in r if i > 3])

In [16]:
bleu_hook.compute_bleu(reference_corpus = rights,
                       translation_corpus = results)

0.00020161743